<a href="https://colab.research.google.com/github/michalis0/DataScience_and_MachineLearning/blob/master/Assignements/Part%204/Assignment_part_four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DSML investigation:

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignement part four
### Due 22.10
#### Identifying the suspects credit score
We received informations that the rogue agent has a good credit score.

Our spies at SIMF have managed to collect financial information relating to our suspects as well as a training dataset.

Create a Neural Network over the training dataset `df` to identify which of the suspects have a good Credit_Mix


## Getting to know our data

* Age: a users age
* Occupation: a users employment field
* Annual_Income: a users annual income
* Monthly_Inh_Salary: the calculated salary received by a given user on a monthly basis
* Num_Bank_Accounts: the number of bank accounts possessed by a given user
* Num_Credit_Cards: the number of credit card given user possesses
* Interest_Rate: The interest rate on those cards (if multiple then its the average)
* Num_of_Loans: The number of loans of each user
* Delay_from_due_date: payment tardiness of user
* Num_of_Delayed_Payment: the count of delayed payments
* Changed_Credit_Limit: NaN
* Num_Credit_Inquiries: NaN
* Credit_Mix: The users credit score
* Outsting_Debt: Outstanding debt
* Credit_Utilization_Ratio: the percentage of borrowed money over borrowing allowance
* Payment_of_Min_Amount: does the user usually pay the minimal amount (categorical)
* Total_EMI_per_month: Monthly repayments to be made
* Amount_invested_monthly: The amout put in an investment fun by the user on a monthly basis
* Payment_Behaviour: the users payment behavior (categorical)
* Monthly_Balance: The users end of the month balance
* AutoLoan: If the user has an active loan for their vehicule
* Credit-BuilderLoan: If the user has a loan to increase their credit score
* DebtConsolidationLoan, HomeEquityLoan, MortgageLoan, NotSpecified, PaydayLoan, PersonalLoan, StudentLoan: different types of loans(categorical features)



In [1]:
# Import required packages
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/train_classification.csv", index_col='Unnamed: 0').dropna()
suspects = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/suspects.csv", index_col='Unnamed: 0').dropna()

In [3]:
df.info()
suspects.info()


<class 'pandas.core.frame.DataFrame'>
Index: 29223 entries, 0 to 49998
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       29223 non-null  int64  
 1   Occupation                29223 non-null  object 
 2   Annual_Income             29223 non-null  float64
 3   Monthly_Inh_Salary        29223 non-null  float64
 4   Num_Bank_Accounts         29223 non-null  int64  
 5   Num_Credit_Card           29223 non-null  int64  
 6   Interest_Rate             29223 non-null  int64  
 7   Num_of_Loan               29223 non-null  int64  
 8   Delay_from_due_date       29223 non-null  int64  
 9   Num_of_Delayed_Payment    29223 non-null  int64  
 10  Changed_Credit_Limit      29223 non-null  float64
 11  Num_Credit_Inquiries      29223 non-null  float64
 12  Credit_Mix                29223 non-null  object 
 13  Outsting_Debt             29223 non-null  float64
 14  Credit_Util

In [4]:
df["Credit_Mix"].value_counts()

Credit_Mix
Standard    13421
Good         8963
Bad          6839
Name: count, dtype: int64

# 1. Preparing the data
## 1.1 Data cleaning
 Perform OHE over the "Occupation" feature

 Then, perform LE over Payment_of_Min_Amount and Payment_Behaviour

 _hint: As we will be testing only one model no need to define a pipeline_

In [5]:

# Your code here:
ohe = OneHotEncoder(sparse= False)
df_fitted = ohe.fit_transform(df[['Occupation']])
categories = ohe.get_feature_names_out(input_features=['Occupation'])
df_encoded = pd.DataFrame(df_fitted, columns=categories, index= df.index)
# Replace the 'Occupation' column with the one-hot encoded columns
df_encoded = pd.concat([df.drop(['Occupation'], axis=1), df_encoded], axis=1)
print(df_encoded.head())
le = LabelEncoder()
print(df['Payment_of_Min_Amount'], df['Payment_Behaviour'])
df_encoded['Payment_of_Min_Amount'] = le.fit_transform(df_encoded['Payment_of_Min_Amount'])
df_encoded['Payment_Behaviour'] = le.fit_transform(df_encoded['Payment_Behaviour'])

print(df_encoded['Payment_of_Min_Amount'], df_encoded['Payment_Behaviour'])

df_encoded.info()




   Age  Annual_Income  Monthly_Inh_Salary  Num_Bank_Accounts  Num_Credit_Card  \
0   23       19114.12         1824.843333                  3                4   
1   24       19114.12         1824.843333                  3                4   
3   24       19114.12         4182.004291                  3                4   
5   28       34847.84         3037.986667                  2                4   
8   35      143162.64         4182.004291                  1                5   

   Interest_Rate  Num_of_Loan  Delay_from_due_date  Num_of_Delayed_Payment  \
0              3            4                    3                       7   
1              3            4                    3                       9   
3              3            4                    4                       5   
5              6            1                    3                       3   
8              8            3                    8                    1942   

   Changed_Credit_Limit  ...  Occupation_Ent

/home/nathan/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## 1.2 Dataset splitting

Split the dataset in two, first X with your independent features and then y with the dependent feature **CreditMix**.

Then perform :
* OneHotEncoding over the **CreditMix** feature.
* A MinMaxScaller over the independent features

In [6]:
# Your code here:
X = pd.DataFrame(df_encoded.drop(columns=['Credit_Mix']))
y = pd.DataFrame(df_encoded['Credit_Mix'])
#print(y.index)
y_fitted = ohe.fit_transform(y[['Credit_Mix']])
categories_credit_mix = ohe.get_feature_names_out(input_features=['Credit_Mix'])
y = pd.DataFrame(y_fitted, columns=categories_credit_mix, index= y.index)
# Replace the 'Occupation' column with the one-hot encoded columns
#y = pd.concat([y.drop(['Credit_mix'], axis=1), y], axis=1)
print(y)

#Define the scaler
scaler = MinMaxScaler()
#Fit the scaler
scaler.fit_transform(X)

X.loc[:,:] = scaler.transform(X)
print(X)

       Credit_Mix_Bad  Credit_Mix_Good  Credit_Mix_Standard
0                 0.0              1.0                  0.0
1                 0.0              1.0                  0.0
3                 0.0              1.0                  0.0
5                 0.0              1.0                  0.0
8                 0.0              1.0                  0.0
...               ...              ...                  ...
49990             0.0              1.0                  0.0
49992             1.0              0.0                  0.0
49993             1.0              0.0                  0.0
49997             0.0              1.0                  0.0
49998             0.0              1.0                  0.0

[29223 rows x 3 columns]
            Age  Annual_Income  Monthly_Inh_Salary  Num_Bank_Accounts  \
0      0.057177       0.000502            0.102087           0.002242   
1      0.057287       0.000502            0.102087           0.002242   
3      0.057287       0.000502     

/home/nathan/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### 1.2.1 Train Test splitting
Now split the data in X_train, X_test, y_train, y_test, 

You can use test_size = 0.2 and a random_state of 42

In [7]:
# Your code here
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(23378, 42) (5845, 42) (23378, 3) (5845, 3)


### 1.2.2 final touches
Convert your datasets to `Torch tensors` of type `torch.float`

In [8]:
#Your code here:
X_train = torch.tensor(X_train.values, dtype=torch.float)
y_train = torch.tensor(y_train.values, dtype=torch.float)
X_test = torch.tensor(X_test.values, dtype=torch.float)
y_test = torch.tensor(y_test.values, dtype=torch.float)
print(X_train.size(), y_train.size(), y_test.size(), X_test.size())


torch.Size([23378, 42]) torch.Size([23378, 3]) torch.Size([5845, 3]) torch.Size([5845, 42])


# 2 Model preparation:

## 2.1 Define a Neural network model and instantiate it.
You can set the number of neurons to 150.

In [9]:
# Define a neural network class here:
class Net(nn.Module):
    def __init__(self, D_in, H1, D_out):
        super(Net, self).__init__()

        self.linear1 = nn.Linear(D_in, H1)        # Linear transformation for hidden layer
        self.linear2 = nn.Linear(H1, D_out)       # Linear transformation for output layer
        self.activation = nn.ReLU()               # Activation function for hidden layer

    def forward(self, x):
        y_pred = self.activation(self.linear1(x))   # Hidden layer: linear transformation + ReLU
        y_pred = self.linear2(y_pred)               # Output layer: linear transformation
        return y_pred

# Define the input and output sizes
D_in =  X_train.shape[1]
D_out =  y_train.shape[1]


In [10]:
# Instantiate your model here
model1 = Net(D_in, 150, D_out)
print(model1)
pytorch_total_params = sum(p.numel() for p in model1.parameters() if p.requires_grad)
print(pytorch_total_params)

Net(
  (linear1): Linear(in_features=42, out_features=150, bias=True)
  (linear2): Linear(in_features=150, out_features=3, bias=True)
  (activation): ReLU()
)
6903


## 2.2 finding the best model:
Identify, amongst the following options the best parameters for your model:

* `criterion` : [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), [BCEWithLogitsLoss](hhttps://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)
* `iterations` : 150, 250, 500
* `learning rate` : 0.00005, 0.001, 12.031


_Hint: restart your runtime between each execution to ensure that previous neural networks dont interfere with your current one_

_You can evaluate your model based on it's accuracy over the test set_

In [11]:
# Define your loss function here:
criterion = torch.nn.BCEWithLogitsLoss()#reduction='sum')
# Define your Adam optimizer for finding the weights of the network here
optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)

In [12]:
# Perform your iterations here
losses1 = []
losses1_test = []

for t in range(500):                # 500 iterations

    # Forward pass: compute prediction on training set
    y_pred_train = model1(X_train)
    y_pred_test = model1(X_test)
    # Compute loss
    loss = criterion(y_pred_train, y_train)
    print(t, loss.item())
    losses1.append(loss.item())
    if torch.isnan(loss):
        break

    # Compute gradient
    optimizer.zero_grad()
    loss.backward()

    # Update
    optimizer.step()

    # Compute loss on test set
    losses1_test.append(criterion(model1(X_test), y_test).item())


0 0.6912212371826172
1 0.6867984533309937
2 0.6825186014175415
3 0.678371787071228
4 0.6743510961532593
5 0.670447051525116
6 0.6666492223739624
7 0.6629490256309509
8 0.6593380570411682
9 0.6558096408843994
10 0.6523579359054565
11 0.6489788293838501
12 0.645669162273407
13 0.6424286961555481
14 0.639256477355957
15 0.6361531019210815
16 0.6331199407577515
17 0.6301591992378235
18 0.6272759437561035
19 0.624475359916687
20 0.6217633485794067
21 0.6191443204879761
22 0.6166223287582397
23 0.6142007112503052
24 0.6118820905685425
25 0.6096687912940979
26 0.6075620055198669
27 0.6055590510368347
28 0.6036562919616699
29 0.601847767829895
30 0.6001246571540833
31 0.5984771251678467
32 0.5968924164772034
33 0.5953552722930908
34 0.5938518047332764
35 0.5923669934272766
36 0.5908860564231873
37 0.5893957614898682
38 0.5878837704658508
39 0.586341142654419
40 0.5847578644752502
41 0.5831266045570374
42 0.5814438462257385
43 0.5797064900398254
44 0.5779138803482056
45 0.5760665535926819
46 0.

## 2.3 Model Accuracy
Identify the models accuracy over the train and test parts of the training dataset

In [13]:
# deactivate dropout layers
model1.eval()
# Training accuracy

# Convert y_train to a NumPy array if it's not already
y_train = y_train.numpy() if isinstance(y_train, torch.Tensor) else y_train

# Make sure both y_pred_train and y_train are 1D arrays of class labels
y_pred_train = y_pred_train.argmax(axis=1) if len(y_pred_train.shape) > 1 else y_pred_train
y_train = y_train.argmax(axis=1) if len(y_train.shape) > 1 else y_train

# Check that the data types are integers
#y_pred_train = y_pred_train.astype(int)
y_train = y_train.astype(int)

# Calculate accuracy
accuracy_train = accuracy_score(y_train, y_pred_train)
print(accuracy_train)

# test accuracy
y_test = y_test.numpy() if isinstance(y_test, torch.Tensor) else y_test

# Make sure both y_pred_train and y_train are 1D arrays of class labels
y_pred_test = y_pred_test.argmax(axis=1) if len(y_pred_test.shape) > 1 else y_pred_test
y_test = y_test.argmax(axis=1) if len(y_test.shape) > 1 else y_test

# Check that the data types are integers
#y_pred_test = y_pred_test.astype(int)
y_test = y_test.astype(int)

# Calculate accuracy
accuracy_test = accuracy_score(y_test, y_pred_test)
print(accuracy_test)

0.8291128411326889
0.8177929854576561


# 3. Predictions over the suspects dataset
## 3.1 Retrain a new model over the full training dataset
#### Please use the following parameters for this section:
* ``neurons`` = 150
* ``learning`` rate = 0.00005
* ``criterion`` = CrossEntropyLoss
* `iterations` = 500

_hint you may have to redo some preprocessing as you did in part one_

In [21]:
#preprocessing: X = training and suspects is testing dataset
#suspects = suspects.drop(columns=['userID'])
print(suspects[''])
print(X.info())

0       1
1       1
3       1
5       1
8       1
       ..
1231    2
1233    2
1235    2
1236    0
1237    2
Name: Payment_of_Min_Amount, Length: 714, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 29223 entries, 0 to 49998
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       29223 non-null  float64
 1   Annual_Income             29223 non-null  float64
 2   Monthly_Inh_Salary        29223 non-null  float64
 3   Num_Bank_Accounts         29223 non-null  float64
 4   Num_Credit_Card           29223 non-null  float64
 5   Interest_Rate             29223 non-null  float64
 6   Num_of_Loan               29223 non-null  float64
 7   Delay_from_due_date       29223 non-null  float64
 8   Num_of_Delayed_Payment    29223 non-null  float64
 9   Changed_Credit_Limit      29223 non-null  float64
 10  Num_Credit_Inquiries      29223 non-null  float64
 11  Outsting_De

In [15]:
class Net(nn.Module):
    def __init__(self, D_in, H1, D_out):
        super(Net, self).__init__()

        self.linear1 = nn.Linear(D_in, H1)        # Linear transformation for hidden layer
        self.linear2 = nn.Linear(H1, D_out)       # Linear transformation for output layer
        self.activation = nn.ReLU()               # Activation function for hidden layer

    def forward(self, x):
        y_pred = self.activation(self.linear1(x))   # Hidden layer: linear transformation + ReLU
        y_pred = self.linear2(y_pred)               # Output layer: linear transformation
        return y_pred

# Define the input and output sizes
D_in =  df_full_train.shape[1]
D_out =  y_train.shape[1]

IndexError: tuple index out of range

In [ ]:
# Define your MSE loss here:
criterion = nn.CrossEntropyLoss()

# Define your Adam optimizer for finding the weights of the network here:


In [ ]:
# perform your training here
losses1 = []
losses1_test = []

for t in range(500):                # 500 iterations

    # Forward pass: compute prediction on training set
    y_pred = model2(df_full_train)

    # Compute loss
    loss = criterion(y_pred, df_full_test)
    print(t, loss.item())
    losses1.append(loss.item())
    if torch.isnan(loss):
        break

    # Compute gradient
    optimizer.zero_grad()
    loss.backward()

    # Update
    optimizer.step()

    # Compute loss on test set
    #losses1_test.append(criterion(model2(X_test), y_test).item())

NameError: name 'model2' is not defined

## 3.2 Predict over the suspects dataset

In [ ]:
# Predict which users have a good credit score here:
